# Simulating from Multiple Surveys

LightCurveLynx can simulate observations from multiple surveys in a single run. In this notebook, we show how to perform these simulations and discuss how it works.

In [ ]:
import numpy as np

from lightcurvelynx.astro_utils.passbands import PassbandGroup
from lightcurvelynx.models.basic_models import ConstantSEDModel
from lightcurvelynx.obstable.opsim import OpSim
from lightcurvelynx.simulate import simulate_lightcurves

# Usually we would not hardcode the path to the passband files, but for this demo we will use a relative path
# to the test data directory so that we do not have to download the files.
table_dir = "../../tests/lightcurvelynx/data/passbands"

## Prerequisite Data

For each survey that we would like to include in the simulation, we need two pieces of information:

  * An `ObsTable` that includes the survey’s pointing and noise information.
  * A `PassbandGroup` for that survey.

Let's start by creating toy data for each survey.

The first survey will include pointings at two locations (0.0, 10.0) and (180.0, -10.0) in the "g" and "r" bands.

In [ ]:
obsdata1 = {
    "time": [0.0, 1.0, 2.0, 3.0],
    "ra": [0.0, 0.0, 180.0, 180.0],
    "dec": [10.0, 10.0, -10.0, -10.0],
    "filter": ["g", "r", "g", "r"],
    "zp": [5.0, 6.0, 7.0, 8.0],
    "seeing": [1.12, 1.12, 1.12, 1.12],
    "skybrightness": [20.0, 20.0, 20.0, 20.0],
    "exptime": [29.2, 29.2, 29.2, 29.2],
    "nexposure": [2, 2, 2, 2],
}
obstable1 = OpSim(obsdata1)

passband_group1 = PassbandGroup.from_preset(
    preset="LSST",
    table_dir=table_dir,
    filters=["g", "r", "i"],
)

The second survey will include pointings at two locations (0.0, 10.0) and (90.0, -10.0) in the "r" and "z" bands. 

In [ ]:
obsdata2 = {
    "time": [0.5, 1.5, 2.5, 3.5],
    "ra": [0.0, 90.0, 0.0, 90.0],
    "dec": [10.0, -10.0, 10.0, -10.0],
    "filter": ["r", "z", "r", "z"],
    "zp": [1.0, 2.0, 3.0, 4.0],
    "seeing": [1.12, 1.12, 1.12, 1.12],
    "skybrightness": [20.0, 20.0, 20.0, 20.0],
    "exptime": [29.2, 29.2, 29.2, 29.2],
    "nexposure": [2, 2, 2, 2],
}
obstable2 = OpSim(obsdata2)

passband_group2 = PassbandGroup.from_preset(
    preset="LSST",
    table_dir=table_dir,
    filters=["r", "z"],
)

Note that, while we use the `OpSim` objects for both `ObsTable`s and we use LSST preset for both passbands, this is only for simplicity of the example (only some of the data files are installed by default).

In most cases users will be interested in combining data from multiple surveys. For example we could simulate a trio of surveys: **Rubin** ( `OpSim` and LSST passbands), **ZTF** (`ZTFObsTable` with ZTF passbands) and Roman (data coming soon).


## Model Creation

Nothing changes in the model creation step. We define a model and its parameters as we would with any other simulation.  Here we use a constant SED model (same value for all times and wavelengths). We place the object at (0.0, 10.0) so it is observed by some of the pointings from each survey.

In [ ]:
model = ConstantSEDModel(brightness=100.0, t0=0.0, ra=0.0, dec=10.0, redshift=0.0, node_label="my_star")

## Simulation

We run the simulation by passing in a *list** of `ObsTable` and a *list* of `PassbandGroup`.  The parameter space is only sampled once for each object, so the observations in each survey are consistent with respect to the parameterization. The times of observation and filters used are determined by each survey. And the bandflux is computed using that survey's passbands.

We can simulate a single object and look at its light curve.

In [ ]:
results = simulate_lightcurves(
    model,
    1,
    [obstable1, obstable2],
    [passband_group1, passband_group2],
    obstable_save_cols=["zp"],
)
print(results["lightcurve"][0])

In addition to the standard information like mjd and flux, we capture the survey index to help us trace which survey the data came from. We also can save per-survey, per-observation data, such as the zero point for that observation.